In [108]:
def calculate_count(card):
    if card in [2,3,4,5,6]:
        count = 1
    elif card in [1,10,11]:
        count = -1
    else:
        count = 0
    
    return count

In [109]:
#Trying to manipulate hands simulation data to run calcs

In [110]:
import pandas as pd
import ast
from statistics import mean

In [111]:
num_samples = 10
df = pd.read_csv('sample.csv')
df.head()

,hand_id,player_starting_hand,dealer_starting_hand,dealer_show_card,player_final_hand(s),dealer_final_hand,result(s),num_remaining_cards
0,1,"[4, 4]","[6, 5]",6,"[[4, 8], [4, 6, 10]]","[6, 5, 8]","['d', 'pdd']",304
1,2,"[10, 10]","[10, 2]",10,"[10, 10]","[10, 2, 7]",p,299
2,3,"[7, 7]","[3, 6]",3,"[[7, 2, 5], [7, 11, 6]]","[3, 6, 2, 2, 10]","['pdd', 'd']",288
3,4,"[7, 10]","[7, 7]",7,"[7, 10]","[7, 7, 10]",p,283
4,5,"[9, 8]","[8, 4]",8,"[9, 8]","[8, 4, 7]",d,278


In [112]:
# Loop through the rows of the DataFrame
flists = []
running_count = 0
prev_remaining_cards = 0
for index, row in df.iterrows():
    if row['num_remaining_cards'] > prev_remaining_cards:
        running_count = 0
    prev_remaining_cards = row['num_remaining_cards']
    player_starting_hand = ast.literal_eval(row['player_starting_hand'])
    if 11 in player_starting_hand:
        soft_status = 'soft'
    else:
        soft_status = 'hard'
    
    hand_id = int(row['hand_id'])
    dealer_show_card = int(row['dealer_show_card'])
    player_starting_sum = sum(player_starting_hand) #--> in df
    dealer_starting_hand = ast.literal_eval(row['dealer_starting_hand'])
    dealer_starting_sum = sum(dealer_starting_hand) #--> in df
    player_final_hands = ast.literal_eval(row['player_final_hand(s)'])
    dealer_final_hand = ast.literal_eval(row['dealer_final_hand'])
    dealer_final_sum = sum(dealer_final_hand)
    hand_result = row['result(s)']
    split_status  = 'no split'
    split_hand_sums = []
    remaining_cards = int(row['num_remaining_cards'])
    remaining_decks = remaining_cards // 52
    true_count = 0
    
    print(player_final_hands)
    for card_or_hand in player_final_hands:
        if isinstance(card_or_hand, int):
            #print(card_or_hand)
            #print("NO SPLIT")
            player_final_sum = sum(player_final_hands) #--> in df
            split_status = 'no split'
            num_hands = 1
            running_count = running_count + calculate_count(card_or_hand)
            print('running count: ', running_count)
            #break
        else:
            #print("SPLIT")
            split_status = 'split'
            split_hand_sums.append(sum(card_or_hand)) #[18,16]
            
            print(card_or_hand)
            for card in card_or_hand:
                running_count = running_count + calculate_count(card)
                print("running count: ", running_count)
            #print(isinstance(card_or_hand, list))

    print(dealer_final_hand)
    for card in dealer_final_hand:
        running_count = running_count + calculate_count(card)
        print("running count: ", running_count)

    if split_status == 'split':
        player_final_sum = mean(split_hand_sums)
        num_hands = len(split_hand_sums)

    #Numeric results
    numeric_result = 0
    if hand_result not in ['p','d','pu','pdd','ddd','pbj']:
        #SPLIT HAND
        hand_result = ast.literal_eval(hand_result)
        #print(hand_result)
        for result in hand_result:
            #print(result)
            if result == 'p':
                numeric_result = numeric_result + 1
            elif result == 'd':
                numeric_result = numeric_result - 1
            elif result == 'pu':
                numeric_result = numeric_result
            elif result == 'pdd':
                numeric_result = numeric_result + 2
            elif result == 'ddd':
                numeric_result = numeric_result -2
            elif result == 'pbj':
                numeric_result = numeric_result + 1.5
            #print(numeric_result)
    
    else:
        #ONE RESULT
        if hand_result == 'p':
            numeric_result = 1
        elif hand_result == 'd':
            numeric_result = -1
        elif hand_result == 'pu':
            numeric_result = 0
        elif hand_result == 'pdd':
            numeric_result = 2
        elif hand_result == 'ddd':
            numeric_result = -2
        elif hand_result == 'pbj':
            numeric_result = 1.5

    if remaining_decks == 0:
        true_count = running_count // 1
    else:
        true_count = running_count // remaining_decks
    
    frow = [hand_id, player_starting_sum, soft_status, split_status, num_hands, dealer_show_card, player_final_sum, dealer_final_sum, numeric_result, remaining_cards, remaining_decks, player_final_hands, dealer_final_hand, running_count, true_count]
    flists.append(frow)

#print(flists)


[[4, 8], [4, 6, 10]]
[4, 8]
running count:  1
running count:  1
[4, 6, 10]
running count:  2
running count:  3
running count:  2
[6, 5, 8]
running count:  3
running count:  4
running count:  4
[10, 10]
running count:  3
running count:  2
[10, 2, 7]
running count:  1
running count:  2
running count:  2
[[7, 2, 5], [7, 11, 6]]
[7, 2, 5]
running count:  2
running count:  3
running count:  4
[7, 11, 6]
running count:  4
running count:  3
running count:  4
[3, 6, 2, 2, 10]
running count:  5
running count:  6
running count:  7
running count:  8
running count:  7
[7, 10]
running count:  7
running count:  6
[7, 7, 10]
running count:  6
running count:  6
running count:  5
[9, 8]
running count:  5
running count:  5
[8, 4, 7]
running count:  5
running count:  6
running count:  6
[5, 10, 3]
running count:  7
running count:  6
running count:  7
[10, 8]
running count:  6
running count:  6
[8, 5, 7]
running count:  6
running count:  7
running count:  7
[10, 4, 2, 6]
running count:  6
running count:  

In [113]:
#Create dataframe with hand info
data = flists
df = pd.DataFrame(data, columns = ['hand_id', 'player_starting_sum','soft_status','split_status','num_hands','dealer_show_card', 'player_final_sum','dealer_final_sum','result(s)', 'num_remaining_cards', 'remaining_decks', 'player_final_hands', 'dealer_final_hand', 'running_count','true_count'])
df.head(10)

,hand_id,player_starting_sum,soft_status,split_status,num_hands,dealer_show_card,player_final_sum,dealer_final_sum,result(s),num_remaining_cards,remaining_decks,player_final_hands,dealer_final_hand,running_count,true_count
0,1,8,hard,split,2,6,16.0,19,1.0,304,5,"[[4, 8], [4, 6, 10]]","[6, 5, 8]",4,0
1,2,20,hard,no split,1,10,20.0,19,1.0,299,5,"[10, 10]","[10, 2, 7]",2,0
2,3,14,hard,split,2,3,19.0,23,1.0,288,5,"[[7, 2, 5], [7, 11, 6]]","[3, 6, 2, 2, 10]",7,1
3,4,17,hard,no split,1,7,17.0,24,1.0,283,5,"[7, 10]","[7, 7, 10]",5,1
4,5,17,hard,no split,1,8,17.0,19,-1.0,278,5,"[9, 8]","[8, 4, 7]",6,1
5,6,15,hard,no split,1,10,18.0,18,0.0,273,5,"[5, 10, 3]","[10, 8]",6,1
6,7,13,hard,no split,1,10,20.0,22,1.0,266,5,"[8, 5, 7]","[10, 4, 2, 6]",9,1
7,8,20,hard,no split,1,10,20.0,22,1.0,261,5,"[10, 10]","[10, 3, 9]",7,1
8,9,19,hard,no split,1,10,19.0,20,-1.0,257,4,"[9, 10]","[10, 10]",4,1
9,10,10,hard,no split,1,3,20.0,17,2.0,250,4,"[5, 5, 10]","[3, 3, 2, 9]",8,2


In [114]:
df.to_csv('sample_format.csv', index = False)
